<a href="https://colab.research.google.com/github/dsivov/MyColabs/blob/main/weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://weaviate.io/developers/weaviate/current/
%pip install weaviate-client
client = weaviate.Client("http://64.71.146.93:8080") # <== if you use Docker-compose

In [ ]:
class_obj = {
    "class": "Test_Class", # <= Change to your class name - it will be your collection
    "description": "Add weak conclusion similarity",
    "vectorizer": "none",
    "properties": [
        {
            "dataType": [
                "int"
            ],
            "description": "Frame_nmb",
            "name": "frame_nmbr",
        },
        {
            "dataType": [
                "string"
            ],
            "description": "movie_id",
            "name": "movie_id"
        },
        {
            "dataType": [
                "string"
            ],
            "description": "url",
            "name": "url"
        },
        {
            "dataType": [
                "string"
            ],
            "description": "caption",
            "name": "caption"
        },
        {
            "dataType": [
                "string"
            ],
            "description": "conclusion",
            "name": "conclusion"
        }
    ]
}

In [ ]:
# add the schema
client.schema.create_class(class_obj)
#get the schema
schema = client.schema.get()
#If you want to delete - uncomment
#client.schema.delete_class()
#print the schema
print(json.dumps(schema, indent=4))

In [ ]:
#Create your vector by any model like:
#vector = st_model.encode(line['intents'], convert_to_tensor=True)
#Create data object
data_object = {"movie_id": line['movie_id'], "frame_nmbr": line['frame_num'], 
    "url": line['url'], "caption": line['candidate'], "intent": line['intents']}
#Insert vector with data 
client.data_object.create(
data_object,
"Test_Class",
# str(uuid_),
vector = vector # supported types are `list`, 'numpy.ndarray`, `torch.Tensor` and `tf.Tensor`. Make sure the length matches with your Weaviate settings.
)    

In [ ]:
def search(query="", limit=3):
    before = time.time()
    vec = text2vec(query)
    vec_took = time.time() - before

    before = time.time()
    near_vec = {"vector": vec}
    res = client \
        .query.get("Post", ["content", "_additional {certainty}"]) \
        .with_near_vector(near_vec) \
        .with_limit(limit) \
        .do()
    search_took = time.time() - before

    print("\nQuery \"{}\" with {} results took {:.3f}s ({:.3f}s to vectorize and {:.3f}s to search)" \
          .format(query, limit, vec_took+search_took, vec_took, search_took))
    for post in res["data"]["Get"]["Post"]:
        print("{:.4f}: {}".format(post["_additional"]["certainty"], post["content"]))
        print('---')
print(result)